In [17]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from textpreprocessor import TextPreprocessor
import scipy.sparse as sp
# Define the path to your data file
data_file_path = "C:\\Users\\Saad\\Desktop\\Study\\Saad Learnings\\Python\\School Python\\Intro to AI Assignments\\Assignments\\Project\\Software_5-core.json"

# Read data from the JSON file into a list of dictionaries
reviews = []
with open(data_file_path, 'r') as file:
    for line in file:
        review = json.loads(line)
        reviews.append(review)

# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(reviews)

# what I need in the data: "overall": 5.0 (stars), "vote": "3" (helpful button), "verified": false (real person)


IndentationError: unindent does not match any outer indentation level (textpreprocessor.py, line 23)

In [ ]:
# Filter out instances where "verified" is false
df_verified = df[df['verified'] == True]

# Preprocess the reviews using the TextPreprocessor
processed_df = TextPreprocessor.preprocess_reviews(df_verified)

# Extract features and target variable
X_text = processed_df["cleanedReviewText"]
X_features = processed_df[["overall", "verified"]]
y = processed_df["vote"]

In [ ]:
# Split the data into training and testing sets
X_text_train, X_text_test, X_features_train, X_features_test, y_train, y_test = train_test_split(
    X_text, X_features, y, test_size=0.2, random_state=42)

# Initialize the text preprocessor
preprocessor = TextPreprocessor()

# Preprocess the training and testing data
X_text_train_processed = preprocessor.preprocess_reviews(X_text_train)
X_text_test_processed = preprocessor.preprocess_reviews(X_text_test)

# Convert text data into numerical features using TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_text_train_tfidf = tfidf_vectorizer.fit_transform(X_text_train_processed['cleanedReviewText'])
X_text_test_tfidf = tfidf_vectorizer.transform(X_text_test_processed['cleanedReviewText'])

# Combine text features with other features
X_train = sp.hstack((X_text_train_tfidf, X_features_train), format='csr')
X_test = sp.hstack((X_text_test_tfidf, X_features_test), format='csr')

# Initialize the SVR model for linear
svr_model = SVR(kernel='linear')

# Train the SVR model on the training data
svr_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = svr_model.predict(X_test)

# Calculate the mean squared error of the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)